In [ ]:
'''匯入套件'''
import requests as req
from bs4 import BeautifulSoup as bs
import json, os, pprint, re

# 隨機取得 User-Agent
'''
# 從外部資料來取得清單，清單預設儲存路徑: /tmp
ua = UserAgent(use_external_data=True)
# 從外部資料來取得清單，儲存在指定路徑
ua = UserAgent(use_external_data=True, cache_path=/home/fake_useragent.json)

更詳細的說明，請見以下網頁:
https://pypi.org/project/fake-useragent/
'''
from fake_useragent import UserAgent
ua = UserAgent(use_external_data=True)

'''放置 金庸小說 metadata 的資訊'''
listData = []

'''金庸小說的網址'''
prefix = 'https://tw.ixdzs.com'
url = prefix + '/author/金庸'

'''設定標頭'''
my_headers = {
    'user-agent': ua.random
}

# 沒有放置 txt 檔的資料夾，就建立起來
folderPath = 'jinyong'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

In [ ]:
'''
自訂函式
'''
# 取得小說的主要連結
def getMainLinks():    
    # 清除 list 內容
    listData.clear()
    
    # 走訪首頁
    res = req.get(url = url, headers = my_headers)
    soup = bs(res.text, "lxml")
    
    # 定義欲取得的小說名稱
    list_novel_names = [
        '倚天屠龍記', '連城訣', '書劍恩仇錄', '碧血劍', '鹿鼎記', 
        '俠客行', '射鵰英雄傳', '雪山飛狐', '飛狐外傳', '天龍八部', 
        '白馬嘯西風', '神鵰俠侶', '越女劍', '鴛鴦刀', '笑傲江湖'
    ]
    
    # 取得每一本指定的小說連結
    for novel_name in list_novel_names:
        # 取得超連結
        a = soup.select_one(f'a[href][title={novel_name}]')
        
        # 取得主要連結相關資訊
        listData.append({
            'title': a['title'], # 或是 a.get_text() 取得 innerText
            'link': prefix + a['href'],
            'sub': [] # 之後會放置每一本小說的章回資訊
        })
        
    # 預覽結果
    pprint.pprint(listData)
    

# 取得所有小說的獨立連結
def getSubLinks():
    # 取得章回列表
    for index in range( len(listData) ):
        # 取得 bid
        bid = re.search(r'\/read\/(\d+)\/', listData[index]['link'])[1]
        
        # post 請求，取得章回列表
        my_data = {'bid': bid}
        res = req.post(url = 'https://tw.ixdzs.com/novel/clist/', data = my_data)
        obj_json = res.json()
        
        # 如果回傳訊息為 200 (這個網站自訂的)
        if obj_json['rs'] == 200:
            # 取得章節連結相關資訊
            for obj_data in obj_json['data']:
                if obj_data['ctype'] != '1':
                    listData[index]['sub'].append({
                        'title': obj_data['title'],
                        'link': prefix + f'/read/{bid}/p{obj_data["ordernum"]}.html',
                        'content': '' # 預留給小說內文
                    })
    
    # 預覽結果
    pprint.pprint(listData)


# 建立金庸小說的 json 檔
def saveJson():
    with open(f"{folderPath}/ixdzs_jinyong_post_requests.json", "w", encoding="utf-8") as file:
        file.write( json.dumps(listData, ensure_ascii=False, indent=4) )

In [ ]:
# 主程式
if __name__ == "__main__":
    getMainLinks()
    getSubLinks()
    saveJson()

# 議題思考
- **這個範例沒有直接抓小說內文，如果是你，該怎麼進去每一個內頁去取得文章？**

```
參考:
for index in range( len(listData) ):
    for idx in range( len(listData[index]['sub']) ):
        res = req.get(url = listData[index]['sub'][idx]['link'])
        ...
```

- **有些連結可能沒有用，例如正文、卷○、後記等，當你用 requests 進去瀏覽時，如何略過？**

```
參考:
res = req.get(url = listData[index]['sub'][idx]['link'])
soup = bs(res.text, "lxml")

# 判斷小說內文的區域是否存在
if len(soup.select('article.page-content')) > 0:
    title = soup.select_one('article.page-content h3').get_text()
    content = soup.select_one('article.page-content section').get_text()
```

- **如果你能取得小說內文，有辦法儲存在對應的 content 當中嗎？有辦法另外將內文獨立儲存在各別的 txt 檔當中嗎？**

```
參考:
content = soup.select_one('article.page-content section').get_text()

# 儲存在對應的 key (content)，作為 value
listData[index]['sub'][idx]['content'] = content

# 寫入檔案
with open(f"{folderPath}/{title}.txt", "w", encoding="utf-8") as file:
    file.write(content)
```